In [2]:
import pandas as pd

wine = pd.read_csv("https://bit.ly/wine_csv_data")
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data,target,test_size=0.2,random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input,train_target,test_size=0.2,random_state=42) # 검증 세트 생성
print(sub_input.shape,val_input.shape)

(4157, 3) (1040, 3)


In [4]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input,val_target))
# => 훈련 세트에 과대적합

0.9971133028626413
0.864423076923077


In [5]:
# 교차 검증
from sklearn.model_selection import cross_validate

scores = cross_validate(dt,train_input,train_target)
print(scores)

{'fit_time': array([0.011168  , 0.00854707, 0.00873113, 0.00855947, 0.00906181]), 'score_time': array([0.00099111, 0.00097108, 0.00096655, 0.00093412, 0.00096202]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [6]:
import numpy as np

print(np.mean(scores['test_score'])) # 검증 점수

0.855300214703487


In [7]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt,train_input,train_target,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [8]:
splitter = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores = cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [9]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1) # 객체 생성
gs.fit(train_input,train_target) # 5*5개 모델 훈련

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [13]:
dt = gs.best_estimator_ # 전체 모델 중 검증 점수 높은 모델들
print(dt.score(train_input,train_target))
print(gs.best_params_) # 최적 매개변수
print(gs.cv_results_['mean_test_score']) # 각 매개변수에서 수행한 교차 검증의 평균 점수

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [14]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [15]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001), # 시작,끝,증가분
          'max_depth':range(5,20,1),
          'min_samples_split':range(2,100,10)
          }

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [19]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [20]:
from scipy.stats import uniform, randint

rgen = randint(0,10)
rgen.rvs(10)

array([9, 7, 9, 3, 0, 7, 4, 4, 8, 6])

In [22]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 96,  91, 112,  92, 102, 117,  96,  95, 114,  85]))

In [23]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.96054459, 0.01377964, 0.84441742, 0.16393076, 0.15161971,
       0.59215457, 0.97614841, 0.54568216, 0.50556191, 0.25019909])

In [24]:
params = {'min_impurity_decrease':uniform(0.0001,0.001),
          'max_depth':randint(20,50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1,25)}

In [28]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42,splitter='random'),params,n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)
print(gs.best_params_) # 최적 매개변수 조합
print(np.max(gs.cv_results_['mean_test_score'])) # 최고 교차 검증 점수

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981


In [29]:
dt = gs.best_estimator_
print(dt.score(test_input,test_target))

0.786923076923077
